<a href="https://colab.research.google.com/github/VladimirApter/ml4se_course_2025/blob/main/homeworks/ml4se_course_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Модель: ChatGPT 5**

**Промпт:**
Нужно решить задачу токенезации строкового представления кода.

На вход нам дается строковое представление фрагмента кода на Python (3.11). Этот код нужно превратить в последовательность целых чисел, то есть представить его в виде последовательности токенов. Для токенезации используй BPE и следующий ряд правил:
1. Каждое ключевое слово Python должно целиком превратиться в отдельный токен.
2. Все переносы строк должны превратиться в отдельный токен.
3. Все значимые пробельные символы должны целиком превращаться в отдельный токен. Значимыми пробельными символами являются пробелы и символы табуляции, которые предназначены для обозначения вложенности в Python.

Напиши программу на Python которая будет обрабатывать строковое представление кода вышеописанным способом.


In [5]:
import re
from collections import defaultdict

class Tokenizer:
    def __init__(self):
        self.vocab = {}
        self.inverse_vocab = {}
        self.next_token_id = 256

        for i in range(256):
            self.vocab[chr(i)] = i
            self.inverse_vocab[i] = chr(i)

    @staticmethod
    def _extract_python_tokens(text):
        patterns = [
            (r'\n', 'NEWLINE'),
            (r'[ \t]+', 'INDENT'),
            (r'\b(and|as|assert|async|await|break|class|continue|def|del|elif|else|except|finally|for|from|global|if|import|in|is|lambda|nonlocal|not|or|pass|raise|return|try|while|with|yield|True|False|None)\b', 'KEYWORD'),
        ]

        tokens = []
        pos = 0
        text_len = len(text)

        while pos < text_len:
            matched = False
            for pattern, token_type in patterns:
                regex = re.compile(pattern)
                match = regex.match(text, pos)
                if match:
                    token = match.group()
                    if token_type in ['KEYWORD', 'NEWLINE', 'INDENT']:
                        tokens.append((token, token_type))
                    else:
                        tokens.extend([(ch, 'CHAR') for ch in token])

                    pos = match.end()
                    matched = True
                    break

            if not matched:
                tokens.append((text[pos], 'CHAR'))
                pos += 1

        return tokens

    @staticmethod
    def _get_stats(tokens):
        pairs = defaultdict(int)
        for i in range(len(tokens) - 1):
            if tokens[i][1] == 'CHAR' and tokens[i+1][1] == 'CHAR':
                pairs[(tokens[i][0], tokens[i+1][0])] += 1
        return pairs

    @staticmethod
    def _merge_pair(tokens, pair):
        new_tokens = []
        i = 0
        while i < len(tokens):
            if (i < len(tokens) - 1 and
                tokens[i][1] == 'CHAR' and tokens[i+1][1] == 'CHAR' and
                tokens[i][0] == pair[0] and tokens[i+1][0] == pair[1]):
                new_token = pair[0] + pair[1]
                new_tokens.append((new_token, 'CHAR'))
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1
        return new_tokens

    def train(self, text, num_merges=100):
        tokens = self._extract_python_tokens(text)

        for _ in range(num_merges):
            pairs = self._get_stats(tokens)
            if not pairs:
                break

            best_pair = max(pairs.items(), key=lambda x: x[1])[0]

            new_token = best_pair[0] + best_pair[1]
            self.vocab[new_token] = self.next_token_id
            self.inverse_vocab[self.next_token_id] = new_token
            self.next_token_id += 1

            tokens = self._merge_pair(tokens, best_pair)

    def tokenize(self, text):
        tokens = self._extract_python_tokens(text)

        changed = True
        while changed:
            changed = False
            for token_str, token_id in sorted(self.vocab.items(),
                                            key=lambda x: len(x[0]),
                                            reverse=True):
                if len(token_str) > 1:
                    new_tokens = []
                    i = 0
                    while i < len(tokens):
                        if (tokens[i][1] == 'CHAR' and
                            i + len(token_str) - 1 < len(tokens) and
                            all(tokens[i+j][1] == 'CHAR' for j in range(len(token_str))) and
                            ''.join(tokens[i+j][0] for j in range(len(token_str))) == token_str):

                            new_tokens.append((token_str, 'CHAR'))
                            i += len(token_str)
                            changed = True
                        else:
                            new_tokens.append(tokens[i])
                            i += 1
                    tokens = new_tokens

        token_ids = []
        for token, token_type in tokens:
            if token_type in ['KEYWORD', 'NEWLINE', 'INDENT']:
                if token not in self.vocab:
                    self.vocab[token] = self.next_token_id
                    self.inverse_vocab[self.next_token_id] = token
                    self.next_token_id += 1
                token_ids.append(self.vocab[token])
            elif token in self.vocab:
                token_ids.append(self.vocab[token])
            else:
                for char in token:
                    token_ids.append(self.vocab[char])

        return token_ids


if __name__ == "__main__":
    python_code_example = """
def factorial(n):
    if n <= 1:
        return 1
    else:
        return n * factorial(n-1)

class Calculator:
    def __init__(self):
        self.result = 0

    def add(self, x):
        self.result += x
        return self.result
"""

    tokenizer = Tokenizer()
    tokenizer.train(python_code_example, num_merges=50)

    tokens = tokenizer.tokenize(python_code_example)

    print("Исходный код:")
    print(python_code_example)
    print("\nТокены:")
    print(tokens)


Исходный код:

def factorial(n):
    if n <= 1:
        return 1
    else:
        return n * factorial(n-1)

class Calculator:
    def __init__(self):
        self.result = 0

    def add(self, x):
        self.result += x
        return self.result


Токены:
[10, 304, 32, 279, 10, 305, 306, 32, 110, 32, 280, 32, 281, 10, 307, 308, 32, 49, 10, 305, 309, 58, 10, 307, 308, 32, 110, 32, 42, 32, 284, 10, 10, 310, 32, 290, 10, 305, 304, 32, 297, 10, 307, 269, 32, 61, 32, 48, 10, 10, 305, 304, 32, 301, 32, 302, 10, 307, 269, 32, 303, 32, 120, 10, 307, 308, 32, 269, 10]
